In [1]:
from pathlib import Path
import optuna
from fastai.tabular.core import cont_cat_split
import pandas as pd
from sklearn import compose, impute, pipeline, preprocessing, model_selection, svm
import numpy as np
from sklearn import metrics
from fastai.tabular.all import *

path = Path("./data")
output_path = Path("./output")

In [2]:
def balanced_log_loss(y_true, y_pred):
    nc = np.bincount(y_true)
    return metrics.log_loss(y_true, y_pred, sample_weight=1 / nc[y_true], eps=1e-15)


class BalancedLogLoss:
    def __init__(self, axis=1, smooth=1., alpha=1., gamma=2.):
        store_attr()
        self.balanced_log_loss = balanced_log_loss
        self.focal_loss = FocalLossFlat(gamma=gamma)
        
    def __call__(self, pred, targ, *args, **kwargs):
        y_true = targ.flatten().cpu().detach().numpy()
        y_pred = pred.cpu().detach().numpy()
        try:
            loss = self.balanced_log_loss(y_true, y_pred)
        except Exception as e:
            raise CancelBatchException()
        
        b_log_loss = tensor(loss, requires_grad=True).to(pred.device)
        focal_loss = self.focal_loss(pred, targ)
        
        return focal_loss * self.alpha + b_log_loss
    
    def decodes(self, x):
        return x.argmax(dim=self.axis)
        
    def activation(self, x):
        return F.softmax(x, dim=self.axis)

In [3]:
from collections import defaultdict

import numpy as np
import pandas as pd
from scipy import stats
from sklearn import compose, ensemble, impute, pipeline, preprocessing, tree

def get_preprocess_pipeline(df, cont_cols, cat_cols, drop_cols):
    """
    Returns a pipeline that performs the following transformations:
    * Standard scaling
    * Log transformation
    * Reciprocal transformation
    * Box-Cox transformation
    * Yeo-Johnson transformation
    * Categorical imputing
    * Semi-constant feature binarization

    Based on the EDA from https://www.kaggle.com/code/mateuszk013/icr-eda-balanced-learning-with-lgbm-xgb/notebook

    :param df: The dataframe to be transformed.
    :type df: pandas.DataFrame
    :param cont_names: The names of the continuous variables.
    :type cont_names: list of str
    :param dep_vars: The names of the dependent variables.
    :type dep_vars: list of str
    """

    # Identify columns that doesn't follow a normal distribution
    # find an appropriate transformation for them to follow a normal distribution
    r2_scores = defaultdict(tuple)

    for feature in cont_cols:
        orig = df[feature].dropna()
        _, (*_, R_orig) = stats.probplot(orig, rvalue=True)
        _, (*_, R_log) = stats.probplot(np.log(orig), rvalue=True)
        _, (*_, R_sqrt) = stats.probplot(np.sqrt(orig), rvalue=True)
        _, (*_, R_reci) = stats.probplot(np.reciprocal(orig), rvalue=True)
        _, (*_, R_boxcox) = stats.probplot(stats.boxcox(orig)[0], rvalue=True)
        _, (*_, R_yeojohn) = stats.probplot(stats.yeojohnson(orig)[0], rvalue=True)
        r2_scores[feature] = (
            R_orig * R_orig,
            R_log * R_log,
            R_sqrt * R_sqrt,
            R_reci * R_reci,
            R_boxcox * R_boxcox,
            R_yeojohn * R_yeojohn,
        )

    r2_scores = pd.DataFrame(
        r2_scores,
        index=("Original", "Log", "Sqrt", "Reciprocal", "BoxCox", "YeoJohnson"),
    ).T

    r2_scores["Winner"] = r2_scores.idxmax(axis=1)

    # Identify columns to be transformed
    no_transform_cols = r2_scores.query("Winner == 'Original'").index
    log_transform_cols = r2_scores.query("Winner == 'Log'").index
    reciprocal_transform_cols = r2_scores.query("Winner == 'Reciprocal'").index
    boxcox_transform_cols = r2_scores.query("Winner == 'BoxCox'").index
    yeojohnson_transform_cols = r2_scores.query("Winner == 'YeoJohnson'").index

    # Identify columns that are constant or semi-constant
    numeric_descr = df.drop(columns=drop_cols).describe().T
    semi_constant_mask = np.isclose(numeric_descr["min"], numeric_descr["50%"])
    semi_constant_descr = numeric_descr[semi_constant_mask]
    semi_const_cols_thresholds = semi_constant_descr["50%"].to_dict()

    # List of columns to be transformed
    semi_const_cols = semi_const_cols_thresholds.keys()
    no_transform_cols = no_transform_cols.drop(semi_const_cols, errors="ignore").to_list()
    log_transform_cols = log_transform_cols.drop(semi_const_cols, errors="ignore").to_list()
    reciprocal_transform_cols = reciprocal_transform_cols.drop(semi_const_cols, errors="ignore").to_list()
    boxcox_transform_cols = boxcox_transform_cols.drop(semi_const_cols, errors="ignore").to_list()
    yeojohnson_transform_cols = yeojohnson_transform_cols.drop(semi_const_cols, errors="ignore").to_list()

    # Transformations
    standard_scaling = (
        preprocessing.StandardScaler(),
        no_transform_cols,
    )
    log_transform = (
        pipeline.make_pipeline(
            preprocessing.FunctionTransformer(func=np.log, feature_names_out="one-to-one"),
            preprocessing.StandardScaler(),
        ),
        log_transform_cols,
    )
    reciprocal_transform = (
        pipeline.make_pipeline(
            preprocessing.FunctionTransformer(func=np.reciprocal, feature_names_out="one-to-one"),
            preprocessing.StandardScaler(),
        ),
        reciprocal_transform_cols,
    )
    boxcox_transform = (
        preprocessing.PowerTransformer(method="box-cox", standardize=True),
        boxcox_transform_cols,
    )
    yeojohnson_transform = (
        preprocessing.PowerTransformer(method="yeo-johnson", standardize=True),
        yeojohnson_transform_cols,
    )

    # Other transformations
    categorical_imputing = (
        pipeline.make_pipeline(
            impute.SimpleImputer(strategy="most_frequent"),
            preprocessing.OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1),
        ),
        cat_cols,  # type: ignore
    )
    semi_const_transforms = [
        (
            pipeline.make_pipeline(
                impute.SimpleImputer(strategy="median"),
                preprocessing.Binarizer(threshold=thresh),
            ),
            [col],
        )
        for col, thresh in semi_const_cols_thresholds.items()
    ]

    return pipeline.make_pipeline(
        compose.make_column_transformer(
            standard_scaling,
            log_transform,
            reciprocal_transform,
            boxcox_transform,
            yeojohnson_transform,
            categorical_imputing,
            *semi_const_transforms,
            remainder="drop",
            verbose_feature_names_out=False,
        ),
        impute.KNNImputer(n_neighbors=10, weights="distance"),
    ).set_output(transform="pandas")

    
# Load data
    
df = pd.read_csv(path / "train.csv", index_col="Id")
dep_vars = ["Class"]

drop_vars = ["EJ"]
df.drop(columns=drop_vars, inplace=True)

train_df, test_df = model_selection.train_test_split(df, test_size=0.4, stratify=df[dep_vars], random_state=33)

cont_names, cat_names = cont_cat_split(df, dep_var=dep_vars)

preprocessor = get_preprocess_pipeline(df, cont_names, cat_names, dep_vars)

# Preprocess training data
X_pre = preprocessor.fit_transform(train_df.drop(columns=dep_vars))
train_df = pd.merge(X_pre, train_df[dep_vars], left_index=True, right_index=True)
X = train_df.drop(columns=dep_vars, errors="ignore")
y = train_df[dep_vars]

# Preprocess test data
X_test_pre = preprocessor.transform(test_df.drop(columns=dep_vars))
test_df = pd.merge(X_test_pre, test_df[dep_vars], left_index=True, right_index=True)
X_test = test_df.drop(columns=dep_vars, errors="ignore")
y_test = test_df[dep_vars]

In [4]:
# Resample
from imblearn.over_sampling import SMOTE


def resample(X, y):
    sampler = SMOTE()
    X_res, y_res = sampler.fit_resample(X, y)
    return X_res, y_res

In [5]:
X_res, y_res = resample(X, y)
train_df_res = pd.merge(X_res, y_res, left_index=True, right_index=True)


# Optimize

In [6]:
layers_map = {
    1: [64, 32, 32, 16, 8],
    2: [64, 32, 32, 16, 8, 4],
    3: [64, 32, 32, 16, 8, 4, 2],
    4: [128, 64, 32, 32, 16, 8, 4, 2],
    5: [128, 64, 32, 32, 16, 8, 4],
    6: [128, 64, 32, 32, 16, 8],
    7: [2048, 1024, 512],
    8: [2048, 1024, 512, 256],
    9: [2048, 1024, 512, 256, 128],
    10: [512, 256, 128, 64, 32, 16],
    11: [256, 128, 64, 32, 16],
}

In [7]:
def objective(trial):
    bs = 12
    epochs = 64
    layers_choice = trial.suggest_categorical("layers_choice", layers_map.keys())
    layers = layers_map[layers_choice]
    trial.set_user_attr("layers", layers)

    config = tabular_config(
        ps=trial.suggest_float("ps", 0.01, 0.5, log=True),
        use_bn=trial.suggest_categorical("use_bn", [True, False]),
        bn_final=trial.suggest_categorical("bn_final", [True, False]),
        bn_cont=trial.suggest_categorical("bn_cont", [True, False]),
        lin_first=trial.suggest_categorical("lin_first", [True, False]),
    )

    dls = TabularDataLoaders.from_df(
        train_df_res,
        y_names=dep_vars,
        y_block=CategoryBlock,
        splitter=RandomSplitter(seed=42),
        bs=bs,
        splits=TrainTestSplitter(
            test_size=0.2,
            stratify=train_df[dep_vars],
        ),
    )

    model = tabular_learner(
        dls,
        loss_func=BalancedLogLoss(gamma=trial.suggest_float("gamma", 2, 5, step=0.5)),
        metrics=[accuracy, skm_to_fastai(balanced_log_loss)],
        layers=layers,
        config=config,
        cbs=[
            EarlyStoppingCallback(min_delta=0.0001, patience=8),
        ],
        wd=trial.suggest_float("wd", 1e-3, 1, log=True),
        wd_bn_bias=trial.suggest_categorical("wd_bn_bias", [True, False]),
    )

    model.fit_one_cycle(epochs)

    ypreds, targs = model.get_preds()

    y_true = targs.flatten().cpu().detach().numpy()
    y_pred = ypreds.cpu().detach().numpy()

    val_loss = balanced_log_loss(y_true, y_pred)

    test_dl = dls.test_dl(test_df)
    ypreds, targs = model.get_preds(dl=test_dl)

    y_true = targs.flatten().cpu().detach().numpy()
    y_pred = ypreds.cpu().detach().numpy()
    test_loss = balanced_log_loss(y_true, y_pred)

    return test_loss, test_loss - val_loss

In [8]:
optuna.delete_study(
    study_name="fastai_resample_wider",
    storage="sqlite:///optuna.db",
)

In [9]:
pruner = optuna.pruners.SuccessiveHalvingPruner()
study = optuna.create_study(
    pruner=pruner,
    directions=["minimize", "minimize"],
    study_name="fastai_resample_wider",
    storage="sqlite:///optuna.db",
    load_if_exists=True,
)
study.optimize(objective, n_trials=250)

[I 2023-08-08 23:47:43,975] A new study created in RDB with name: fastai_resample_wider


epoch,train_loss,valid_loss,accuracy,balanced_log_loss,time
0,1.636159,0.585509,0.860656,4.812813,00:00
1,1.996498,3.023148,0.893443,3.680388,00:00
2,2.319857,0.764050,0.975410,0.849334,00:00
3,1.698933,0.756157,0.975410,0.849334,00:00
4,1.367316,1.247948,0.959016,1.415550,00:00
5,0.785213,0.601200,0.975410,0.849334,00:00
6,0.701125,0.447559,0.983607,0.566223,00:00
7,0.397504,1.001770,0.959016,1.415556,00:00
8,0.830507,1.553886,0.942623,1.981779,00:00
9,0.727827,1.131011,0.967213,1.132445,00:00


No improvement since epoch 11: early stopping


[I 2023-08-08 23:47:55,235] Trial 0 finished with values: [0.5042717733654292, 0.41513334794852813] and parameters: {'bn_cont': False, 'bn_final': False, 'gamma': 5.0, 'layers_choice': 8, 'lin_first': True, 'ps': 0.23693243204101042, 'use_bn': False, 'wd': 0.03457123561542312, 'wd_bn_bias': True}. 


epoch,train_loss,valid_loss,accuracy,balanced_log_loss,time
0,0.866366,0.865750,0.516393,17.269388,00:00
1,0.865712,0.864445,0.590164,14.635075,00:00
2,0.864317,0.860308,0.803279,7.024836,00:00
3,0.858016,0.836518,0.909836,3.163983,00:00
4,0.880581,0.476115,0.909836,3.089671,00:00
5,0.871060,0.451707,0.934426,2.192989,00:00
6,0.900078,0.425033,0.959016,1.463507,00:00
7,0.621981,0.409183,0.967213,1.133650,00:00
8,0.803271,0.401024,0.942623,2.030333,00:00
9,0.530596,0.968939,0.942623,1.937443,00:00


...

[I 2023-08-09 00:17:48,394] Trial 133 finished with values: [0.6931969257787739, 4.440892098500626e-16] and parameters: {'bn_cont': True, 'bn_final': True, 'gamma': 2.5, 'layers_choice': 4, 'lin_first': False, 'ps': 0.11804247171805198, 'use_bn': False, 'wd': 0.20345287103123397, 'wd_bn_bias': False}. 


epoch,train_loss,valid_loss,accuracy,balanced_log_loss,time
0,7.428829,2.541455,0.778689,7.771325,00:00
1,5.685952,2.185745,0.901639,3.643713,00:00
2,4.940403,2.458188,0.893443,4.232041,00:00
3,4.460518,4.456897,0.852459,5.978004,00:00
4,4.280441,4.806876,0.844262,6.310115,00:00
5,3.942780,5.201530,0.819672,7.306449,00:00
6,3.616009,1.937963,0.893443,4.317447,00:00
7,3.262275,2.944158,0.860656,5.645892,00:00
8,3.374226,1.302006,0.942623,2.239373,00:00
9,3.088497,1.437599,0.909836,3.653224,00:00


No improvement since epoch 18: early stopping


[I 2023-08-09 00:18:03,907] Trial 134 finished with values: [0.4862595738718482, 0.0692449148254804] and parameters: {'bn_cont': True, 'bn_final': False, 'gamma': 3.5, 'layers_choice': 8, 'lin_first': False, 'ps': 0.45288215842900187, 'use_bn': True, 'wd': 0.20345287103123397, 'wd_bn_bias': False}. 
[W 2023-08-09 00:18:15,092] Trial 135 failed with parameters: {'bn_final': True, 'layers_choice': 8, 'ps': 0.015765123769218355, 'use_bn': True} because of the following error: StorageInternalError('An exception is raised during the commit. This typically happens due to invalid data in the commit, e.g. exceeding max length. ').
Traceback (most recent call last):
  File "/home/vinoth/.pyenv/versions/3.11.2/envs/kaggle/lib/python3.11/site-packages/sqlalchemy/engine/base.py", line 1132, in _commit_impl
    self.engine.dialect.do_commit(self.connection)
  File "/home/vinoth/.pyenv/versions/3.11.2/envs/kaggle/lib/python3.11/site-packages/sqlalchemy/engine/default.py", line 694, in do_commit
    

StorageInternalError: An exception is raised during the commit. This typically happens due to invalid data in the commit, e.g. exceeding max length. 